**Troubleshooting VEP Output and Filtering**

The VEP filtering procedure performed before came up with much less variants than expected for the entire human proteome. We need to find out some reason why this could be the case.

In [12]:
import os
import sys
import pandas as pd
from IPython.display import display

ROOT_DIR = os.environ.get('PYTHONPATH')

TROUBLESHOOT_PATH = os.path.join(ROOT_DIR, 'files/vep_troubleshoot')

VEP_OUTPUT_PATH = os.path.join(TROUBLESHOOT_PATH, 'uniprot_human_variants_vep_all.csv')

# keep dataframe in memory
vep_df = pd.read_csv(VEP_OUTPUT_PATH, sep='\t')

/var/folders/y3/mxnsyg_910x11q_t3z54wkbr0000gn/T/ipykernel_33917/3535052536.py:13: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  vep_df = pd.read_csv(VEP_OUTPUT_PATH, sep='\t')


In [3]:
display(vep_df)

,#Uploaded_variation,SWISSPROT,REF_ALLELE,Allele,Consequence,Protein_position,Amino_acids,CLIN_SIG,gnomADg_AF,CANONICAL
0,NC_000002.12:g.88978974G>A,A0A075B6H7.48,G,A,missense_variant,4,P/L,-,-,YES
1,NC_000002.12:g.88978975G>T,A0A075B6H7.48,G,T,missense_variant,4,P/T,-,-,YES
2,NC_000002.12:g.88978971G>A,A0A075B6H7.48,G,A,missense_variant,5,A/V,-,2.033e-05,YES
3,NC_000002.12:g.88978972C>T,A0A075B6H7.48,C,T,missense_variant,5,A/T,-,6.774e-06,YES
4,NC_000002.12:g.88978968T>G,A0A075B6H7.48,T,G,missense_variant,6,Q/P,-,6.773e-06,YES
...,...,...,...,...,...,...,...,...,...,...
10681953,NC_000001.11:g.158999996T>C,W6CW81.33,T,C,missense_variant,106,H/R,-,2.63e-05,YES
10681954,NC_000001.11:g.158999987G>C,W6CW81.33,G,C,missense_variant,109,S/C,-,-,YES
10681955,NC_000001.11:g.158999987G>T,W6CW81.33,G,T,missense_variant,109,S/Y,-,1.314e-05,YES
10681956,NC_000001.11:g.158999978G>A,W6CW81.33,G,A,missense_variant,112,A/V,-,-,YES


**Try to get rows with benign/pathogenic disregarding ambiguity**

In [8]:
benignRegex = r'(benign)'
pathogenicRegex = r'(pathogenic|damaging)'
consequenceRegex = r'(missense_variant)'

vep_all_benign_df = vep_df[vep_df['CLIN_SIG'].str.contains(benignRegex, regex=True) & vep_df['Consequence'].str.contains(consequenceRegex, regex=True)]
vep_all_pathogenic_df = vep_df[vep_df['CLIN_SIG'].str.contains(pathogenicRegex, regex=True) & vep_df['Consequence'].str.contains(consequenceRegex, regex=True)]

display(vep_all_benign_df)
display(vep_all_pathogenic_df)

/var/folders/y3/mxnsyg_910x11q_t3z54wkbr0000gn/T/ipykernel_33917/1594980902.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  vep_all_benign_df = vep_df[vep_df['CLIN_SIG'].str.contains(benignRegex, regex=True) & vep_df['Consequence'].str.contains(consequenceRegex, regex=True)]
/var/folders/y3/mxnsyg_910x11q_t3z54wkbr0000gn/T/ipykernel_33917/1594980902.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  vep_all_pathogenic_df = vep_df[vep_df['CLIN_SIG'].str.contains(pathogenicRegex, regex=True) & vep_df['Consequence'].str.contains(consequenceRegex, regex=True)]


,#Uploaded_variation,SWISSPROT,REF_ALLELE,Allele,Consequence,Protein_position,Amino_acids,CLIN_SIG,gnomADg_AF,CANONICAL
12190,NC_000002.12:g.210023010T>C,A0AUZ9.110,T,C,missense_variant,968,D/G,benign,0.0006836,YES
12192,NC_000002.12:g.210023010T>C,A0AUZ9.110,T,C,missense_variant,968,D/G,benign,0.0006836,YES
15535,NC_000011.10:g.19234818A>G,A0AVK6.137,A,G,missense_variant,231,I/T,likely_benign,0.0001445,YES
15536,NC_000011.10:g.19234818A>G,A0AVK6.137,A,G,missense_variant,231,I/T,likely_benign,0.0001445,YES
15623,NC_000011.10:g.19230828C>G,A0AVK6.137,C,G,missense_variant,358,S/T,benign,0.0122,YES
...,...,...,...,...,...,...,...,...,...,...
10680783,NC_000010.11:g.119898888C>T,Q9Y6Y8.175,C,T,missense_variant,209,P/S,benign,0.008413,YES
10681168,NC_000010.11:g.119926067C>T,Q9Y6Y8.175,C,T,missense_variant,718,A/V,benign,0.002959,YES
10681169,NC_000010.11:g.119926067C>T,Q9Y6Y8.175,C,T,missense_variant,718,A/V,benign,0.002959,YES
10681650,NC_000008.11:g.119105892C>T,Q9Y6Z7.155,C,T,missense_variant,179,R/W,likely_benign,0.001565,YES


,#Uploaded_variation,SWISSPROT,REF_ALLELE,Allele,Consequence,Protein_position,Amino_acids,CLIN_SIG,gnomADg_AF,CANONICAL
9703,NC_000014.9:g.23104829G>A,A0A1B0GTW7.27,G,A,missense_variant,31,S/F,pathogenic,0.001065,YES
9922,NC_000014.9:g.23102250G>A,A0A1B0GTW7.27,G,A,missense_variant,384,S/L,pathogenic,1.971e-05,YES
14558,NC_000007.14:g.139161018A>G,A0AVF1.135,A,G,missense_variant,263,N/S,"likely_pathogenic,pathogenic",-,YES
14559,NC_000007.14:g.139161018A>G,A0AVF1.135,A,G,missense_variant,263,N/S,"likely_pathogenic,pathogenic",-,YES
22222,NC_000007.14:g.122303281G>A,A0PJY2.135,G,A,missense_variant,278,H/Y,pathogenic,-,YES
...,...,...,...,...,...,...,...,...,...,...
10680087,NC_000001.11:g.7677682C>T,Q9Y6Y1.171,C,T,missense_variant,955,R/W,likely_pathogenic,-,YES
10680187,NC_000001.11:g.7736507A>G,Q9Y6Y1.171,A,G,missense_variant,1077,Y/C,likely_pathogenic,-,YES
10680188,NC_000001.11:g.7736507A>G,Q9Y6Y1.171,A,G,missense_variant,1077,Y/C,likely_pathogenic,-,YES
10681645,NC_000008.11:g.119105885C>G,Q9Y6Z7.155,C,G,missense_variant,176,C/W,pathogenic,-,YES


**Check how many rows reduced when you exclude ambiguous mutations**

In [23]:
import re

ambiguousRegex = r'(uncertain|conflicting)'

vep_unamb_benign_df = vep_all_benign_df[~vep_all_benign_df['CLIN_SIG'].str.contains(ambiguousRegex, regex=True)]
vep_unamb_pathogenic_df = vep_all_pathogenic_df[~vep_all_pathogenic_df['CLIN_SIG'].str.contains(ambiguousRegex, regex=True)]

display(vep_unamb_benign_df)
display(vep_unamb_pathogenic_df)

BENIGN_OUTFILE = os.path.join(TROUBLESHOOT_PATH, 'vep_benign_variants.csv')
PATHOG_OUTFILE = os.path.join(TROUBLESHOOT_PATH, 'vep_pathogenic_variants.csv')

vep_unamb_benign_df.to_csv(BENIGN_OUTFILE, sep='\t', index=False)
vep_unamb_pathogenic_df.to_csv(PATHOG_OUTFILE, sep='\t', index=False)

/var/folders/y3/mxnsyg_910x11q_t3z54wkbr0000gn/T/ipykernel_33917/3878221960.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  vep_unamb_benign_df = vep_all_benign_df[~vep_all_benign_df['CLIN_SIG'].str.contains(ambiguousRegex, regex=True)]
/var/folders/y3/mxnsyg_910x11q_t3z54wkbr0000gn/T/ipykernel_33917/3878221960.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  vep_unamb_pathogenic_df = vep_all_pathogenic_df[~vep_all_pathogenic_df['CLIN_SIG'].str.contains(ambiguousRegex, regex=True)]


,#Uploaded_variation,SWISSPROT,REF_ALLELE,Allele,Consequence,Protein_position,Amino_acids,CLIN_SIG,gnomADg_AF,CANONICAL
12190,NC_000002.12:g.210023010T>C,A0AUZ9.110,T,C,missense_variant,968,D/G,benign,0.0006836,YES
12192,NC_000002.12:g.210023010T>C,A0AUZ9.110,T,C,missense_variant,968,D/G,benign,0.0006836,YES
15535,NC_000011.10:g.19234818A>G,A0AVK6.137,A,G,missense_variant,231,I/T,likely_benign,0.0001445,YES
15536,NC_000011.10:g.19234818A>G,A0AVK6.137,A,G,missense_variant,231,I/T,likely_benign,0.0001445,YES
15623,NC_000011.10:g.19230828C>G,A0AVK6.137,C,G,missense_variant,358,S/T,benign,0.0122,YES
...,...,...,...,...,...,...,...,...,...,...
10680783,NC_000010.11:g.119898888C>T,Q9Y6Y8.175,C,T,missense_variant,209,P/S,benign,0.008413,YES
10681168,NC_000010.11:g.119926067C>T,Q9Y6Y8.175,C,T,missense_variant,718,A/V,benign,0.002959,YES
10681169,NC_000010.11:g.119926067C>T,Q9Y6Y8.175,C,T,missense_variant,718,A/V,benign,0.002959,YES
10681650,NC_000008.11:g.119105892C>T,Q9Y6Z7.155,C,T,missense_variant,179,R/W,likely_benign,0.001565,YES


,#Uploaded_variation,SWISSPROT,REF_ALLELE,Allele,Consequence,Protein_position,Amino_acids,CLIN_SIG,gnomADg_AF,CANONICAL
9703,NC_000014.9:g.23104829G>A,A0A1B0GTW7.27,G,A,missense_variant,31,S/F,pathogenic,0.001065,YES
9922,NC_000014.9:g.23102250G>A,A0A1B0GTW7.27,G,A,missense_variant,384,S/L,pathogenic,1.971e-05,YES
14558,NC_000007.14:g.139161018A>G,A0AVF1.135,A,G,missense_variant,263,N/S,"likely_pathogenic,pathogenic",-,YES
14559,NC_000007.14:g.139161018A>G,A0AVF1.135,A,G,missense_variant,263,N/S,"likely_pathogenic,pathogenic",-,YES
22222,NC_000007.14:g.122303281G>A,A0PJY2.135,G,A,missense_variant,278,H/Y,pathogenic,-,YES
...,...,...,...,...,...,...,...,...,...,...
10680087,NC_000001.11:g.7677682C>T,Q9Y6Y1.171,C,T,missense_variant,955,R/W,likely_pathogenic,-,YES
10680187,NC_000001.11:g.7736507A>G,Q9Y6Y1.171,A,G,missense_variant,1077,Y/C,likely_pathogenic,-,YES
10680188,NC_000001.11:g.7736507A>G,Q9Y6Y1.171,A,G,missense_variant,1077,Y/C,likely_pathogenic,-,YES
10681645,NC_000008.11:g.119105885C>G,Q9Y6Z7.155,C,G,missense_variant,176,C/W,pathogenic,-,YES


**Exclude Non-reviewed Proteins**

In [25]:
REVIEWED_UNIPROT_PATH = os.path.join(ROOT_DIR, 'files/uniprot_human_reviewed_lt_2700aa.csv')

def std_uniprot(uniprot: str):

   pattern = r'\..*'
   
   return uniprot.replace(pattern, '', regex=True)

# standardize SWISSPROT column
vep_unamb_benign_df['SWISSPROT'] = std_uniprot(vep_unamb_benign_df['SWISSPROT'])
vep_unamb_pathogenic_df['SWISSPROT'] = std_uniprot(vep_unamb_pathogenic_df['SWISSPROT'])

display(vep_unamb_benign_df)
display(vep_unamb_pathogenic_df)

/var/folders/y3/mxnsyg_910x11q_t3z54wkbr0000gn/T/ipykernel_33917/3256544208.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vep_unamb_benign_df['SWISSPROT'] = std_uniprot(vep_unamb_benign_df['SWISSPROT'])


,#Uploaded_variation,SWISSPROT,REF_ALLELE,Allele,Consequence,Protein_position,Amino_acids,CLIN_SIG,gnomADg_AF,CANONICAL
12190,NC_000002.12:g.210023010T>C,A0AUZ9,T,C,missense_variant,968,D/G,benign,0.0006836,YES
12192,NC_000002.12:g.210023010T>C,A0AUZ9,T,C,missense_variant,968,D/G,benign,0.0006836,YES
15535,NC_000011.10:g.19234818A>G,A0AVK6,A,G,missense_variant,231,I/T,likely_benign,0.0001445,YES
15536,NC_000011.10:g.19234818A>G,A0AVK6,A,G,missense_variant,231,I/T,likely_benign,0.0001445,YES
15623,NC_000011.10:g.19230828C>G,A0AVK6,C,G,missense_variant,358,S/T,benign,0.0122,YES
...,...,...,...,...,...,...,...,...,...,...
10680783,NC_000010.11:g.119898888C>T,Q9Y6Y8,C,T,missense_variant,209,P/S,benign,0.008413,YES
10681168,NC_000010.11:g.119926067C>T,Q9Y6Y8,C,T,missense_variant,718,A/V,benign,0.002959,YES
10681169,NC_000010.11:g.119926067C>T,Q9Y6Y8,C,T,missense_variant,718,A/V,benign,0.002959,YES
10681650,NC_000008.11:g.119105892C>T,Q9Y6Z7,C,T,missense_variant,179,R/W,likely_benign,0.001565,YES
